# Day 1 Meta and Full Predictions of Pokemon Worlds 2025

If you have not seen it yet, please go checkout the first notebook, [worlds2025_deck_sim](https://github.com/dlf57/pkmnworlds25_deck_sim/blob/main/worlds2025_deck_sim.ipynb).

### Day 1 Meta
So there were a lot of changes with the day 1 meta compared to what we had seen online. Gholdengo is occuping nearly 23% of the meta, so this changes results drastically. Below are the Day 1 meta that were shown on stream. We are going to represent teams not shown on the graphic as 1% because at this time we do not know just how many of these decks were played.

|Deck|Day 1 Share|
|--|--|
|gholdengo-ex|22.9|
|dragapult-dusknoir|17.2|
|gardevoir-ex-sv|13.0|
|raging-bolt-ogerpon|10.1|
|grimmsnarl-froslass|4.9|
|flareon-noctowl|4.4|


### Lets Get into it!
With that being the meta, let's rerun our simulations and see what comes out on top!

In [1]:
import pandas as pd
import numpy as np
import random

### Load Meta & Matchups

We are still going to be using the matchups from the top 16 online tournaments but now looking at the day 1 meta.

In [2]:
# Load csv files
matchups = pd.read_csv("matchups-top16.csv")
meta = pd.read_csv("meta-worlds-day1.csv")

# Create a meta dictionary for Monte Carlo to randomly choose a deck from
meta['share'] = meta['share'] / 100.0
meta_dict = dict(zip(meta['deck'], meta['share']))

### Setup our Win Probability Matrix P[Deck1,Deck2]

To build out the win probability matrix we cannot just take the `win_rate` as that would be looking at how likely the deck is to win just one singular match. Becasue the format is best of 3, we need to find the match win probability for this scenario. 

What are our possible win paths? (where $p$ is our win rate)
 1. WW  - $P_{\text{WW}} = p \times p = p^2$
 2. WLW - $P_{\text{WLW}} = p \times (1-p) \times p = p^2 (1-p)$
 3. LWW - $P_{\text{LWW}} = (1-p) \times p \times p = p^2 (1-p)$

Now we sum these scenarios to be able to calculate the probability of a match win $P_{\text{match win}}$:

$P_{\text{match win}} = P_{\text{WW}} + P_{\text{WLW}} + P_{\text{LWW}}$

$P_{\text{match win}} = p^2 + 2p^2(1-p)$

$P_{\text{match win}} = p^2 \big( 1 + 2(1-p) \big)$

$P_{\text{match win}} = p^2 \big( 3 - 2p \big)$

In scenarios where decks have not faced eachother, we are going to make the assumption that the match will be 50/50. While not realistic, it is the best we can do with the information at this time

**CAVEAT** - Ties are possible in the TCG. For the purpose of this simulation we will be treating ties as a loss. For Worlds, this is not that detrimental as there are set numbers of wins needed to get through Day 1 and it is very unlikely to win out the tournament with multiple ties.

In [3]:
# Get list of all decks
decks = sorted(set(matchups['deck1']).union(matchups['deck2']))
deck_index = {d:i for i,d in enumerate(decks)}
n = len(decks)

# Build win probability matrix P[a,b] = P(a beats b)
P = np.zeros((n,n))

for _, row in matchups.iterrows():
    a = deck_index[row['deck1']]
    b = deck_index[row['deck2']]

    # If total is NaN or zero, treat as no data
    total = row['total'] if not pd.isna(row['total']) else 0
    wins = row['wins'] if not pd.isna(row['wins']) else 0

    if total == 0:
        # No games → fall back to 50%
        p_match = 0.5
    else:
        # Calculate the match win probability
        p_win = wins / total
        p_match = p_win**2 * (3 - 2*p_win)

    P[a,b] = p_match

# mirror matches will always be 50%
for i in range(n):
    P[i,i] = 0.5

### Before Simulating, Let's Get an Idea of how Decks do Overall Meta

In [4]:
def matchup_vs(deck_name, meta_dist, n):
    i = deck_index[deck_name]
    total = 0
    for opp, share in meta_dist.items():
        j = deck_index[opp]
        total += P[i,j] 
    return total /n

vs_meta = []
for deck in decks:
    win_pct = matchup_vs(deck, meta_dict, n) * 100
    vs_meta.append((deck, win_pct))

df_vs_meta = pd.DataFrame(vs_meta, columns=['Deck','Win % vs Meta'])
df_vs_meta.sort_values('Win % vs Meta', ascending=False)

,Deck,Win % vs Meta
11,grimmsnarl-froslass,51.066469
0,charizard-dusknoir,50.409488
4,dragapult-dusknoir,49.265126
1,charizard-pidgeot,48.854753
9,gardevoir-ex-sv,48.607443
3,dragapult-charizard,47.428072
2,crustle-dri,47.122858
6,ethan-typhlosion,45.946399
5,dragapult-ex,42.221345
8,froslass-munkidori,39.918912


### Day 1

Our next step is to simulate Day 1 and return what the Day 2 meta will look like

In [5]:
def simulate_day1(deck_name, meta_dist, day1_rounds=8, min_wins=6, sims=100000):
    decks_list, fracs = zip(*meta_dist.items())
    fracs = np.array(fracs) / sum(fracs)
    qualify_count = 0
    
    for _ in range(sims):
        wins = 0
        for _ in range(day1_rounds):
            opp = random.choices(decks_list, weights=fracs, k=1)[0]
            pwin = P[deck_index[deck_name], deck_index[opp]]
            if random.random() < pwin:
                wins += 1
        if wins >= min_wins:
            qualify_count += 1
    return qualify_count / sims


# Simulate the probability of Day 2 conversion
day2_probs = {}
for deck in decks:
    day2_probs[deck] = simulate_day1(deck, meta_dict)

# Get they Day 2 meta from Day 2 conversion probabilities
day2_meta = {}
for deck in decks:
    day2_meta[deck] = meta_dict.get(deck, 0) * day2_probs[deck]

# Normalize to sum to 1
total_share = sum(day2_meta.values())
if total_share > 0:
    for deck in day2_meta:
        day2_meta[deck] /= total_share

# Convert to pandas for visualization
df_day2 = pd.DataFrame(list(day2_meta.items()), columns=['Deck', 'Meta Share'])
df_day2['Meta Share'] = df_day2['Meta Share'] * 100
df_day2.sort_values(by=['Meta Share'], ascending=False)

,Deck,Meta Share
10,gholdengo-ex,32.352809
4,dragapult-dusknoir,21.425081
9,gardevoir-ex-sv,12.560087
11,grimmsnarl-froslass,10.216153
14,raging-bolt-ogerpon,6.884901
7,flareon-noctowl,5.204591
0,charizard-dusknoir,2.338323
3,dragapult-charizard,2.093864
6,ethan-typhlosion,1.602683
13,joltik-box,1.590770


### Day 2, Top 16, & Champion

Just like we did with Day 1, we are going to simulate Day 2, Top 16, and Win chance. In 2024, Day 2 consisted of 4 rounds of swiss with all 10+ win records continuing on to Top 16. We are going to use the same number of rounds of swiss, but with needing to get 3 wins to make Top 16. 

In [6]:
def simulate_day2(deck_name, meta_dist, swiss_rounds=4, cut_wins=3, sims=100000):
    top16_count = 0
    win_event_count = 0
    decks_list, fracs = zip(*meta_dist.items())

    for _ in range(sims):
        wins = 0
        for _ in range(swiss_rounds):
            opp = random.choices(decks_list, weights=fracs, k=1)[0]
            pwin = P[deck_index[deck_name], deck_index[opp]]
            if random.random() < pwin:
                wins += 1

        made_cut = wins >= cut_wins
        if made_cut:
            top16_count += 1
            # Simulate Top 16 (4 rounds)
            for _ in range(4):
                opp = random.choices(decks_list, weights=fracs, k=1)[0]
                pwin = P[deck_index[deck_name], deck_index[opp]]
                if random.random() >= pwin:
                    break
            else:
                win_event_count += 1

    return top16_count / sims, win_event_count / sims


results = []
for d in decks:
    p_cut, p_win = simulate_day2(d, day2_meta)
    r = {}
    r.update({'Deck': d})
    r.update({'Top 16%': p_cut})
    r.update({'Win %': p_win})
    results.append(r)

df_results = pd.DataFrame(results)
df_results['Top 16%'] = df_results['Top 16%'] * 100
df_results['Win %'] = df_results['Win %'] * 100
df_results.sort_values(by=['Win %'], ascending=False)

,Deck,Top 16%,Win %
0,charizard-dusknoir,36.943,3.053
2,crustle-dri,33.279,2.396
7,flareon-noctowl,32.851,2.299
3,dragapult-charizard,30.138,1.873
6,ethan-typhlosion,29.448,1.771
13,joltik-box,30.048,1.754
11,grimmsnarl-froslass,30.064,1.723
10,gholdengo-ex,28.584,1.545
4,dragapult-dusknoir,24.824,1.090
5,dragapult-ex,23.987,0.951


### Tournament Outcome

So these are quite different results than what we saw before. Charizard/Dusknoir becomes a very solid deck for those playing (as was seen on stream actually beating a Crustle). Crustle is additionally a solid pick. While I do not think my intial predictions were all that wrong as Dragapult/Charizard is still up there, the large field of Gholdengo made decks like Flareon/Noctowl and Ethan's Typhlosion much better calls than they initially had been. 

I will be interested to see what the meta breakdown of what makes it to Day 2 and will be looking forward to simulating that tomorrow!